# Homework 3

**Name:** Oscar Eduardo Arámbula Vega

**e-mail:** [oscar.arambula4388@alumnos.udg.mx](mailto:oscar.arambula4388@alumnos.udg.mx)

## Minimum requirements:

- **Functions**: Three types of RWs and three metrics
- **Trajectory selector**: Radio buttons (or similar) to select the type of trajectory
- **Dynamic parameters menu**: Display trajectory parameters according to the selected trajectory
- **Metric selector**: Drop-down menu (or similar) to select metric to display
- **Plot**: Trajectory (3D projection) and curve or histogram of the selected metric

## Random walks and metrics

I ported the previous random walks and metrics from past homework to this one. But instead of defining it in the Jupyter Notebook,
I create a python package that will hold the implementations, and here I just import them.

## Modules



In [1]:
from typing import Literal, get_args

import numpy as np
import pandas as pd
import panel as pn
import panel.widgets as pnw
import plotly.graph_objects as go
import random_walk as walk
import param

pn.extension("plotly")

In [2]:
# Types
TrajectoryType = Literal["BM", "CRW", "LF"]

MetricType = Literal["Path Length", "Mean Square Displacements", "Turning Angles"]


# Widgets
WalkSelectorWidget = pnw.RadioButtonGroup(
    name="Trajectory selector",
    options=list(get_args(TrajectoryType)),
    button_type="primary",
    button_style="outline",
)

StepsWidget = pnw.IntInput(name="Steps", value=100, step=100, start=50, end=10000)

XPositionWidget = pnw.IntInput(name="X", value=0, step=1)
YPositionWidget = pnw.IntInput(name="Y", value=0, step=1)

SpeedWidget = pnw.IntInput(name="Speed", value=1, step=1, start=1)

CauchyCoefficientWidget = pnw.FloatSlider(
    name="Cauchy", value=0.7, step=0.05, start=0.05, end=0.95
)
AlphaLevyWidget = pnw.FloatSlider(
    name="Alpha Levy", value=2, step=0.05, start=0.05, end=2
)

MetricSelectorWidget = pnw.Select(name="Metric", options=list(get_args(MetricType)))

CommonWidgets = StepsWidget, XPositionWidget, YPositionWidget, SpeedWidget

TrajectoryForm: dict[TrajectoryType, pn.viewable] = {
    "BM": pn.Card(*CommonWidgets, title="Brownian Motion", collapsible=False),
    "CRW": pn.Card(
        *CommonWidgets,
        pn.layout.Divider(),
        CauchyCoefficientWidget,
        title="Correlated Random Walk",
        collapsible=False,
    ),
    "LF": pn.Card(
        *CommonWidgets,
        pn.layout.Divider(),
        CauchyCoefficientWidget,
        AlphaLevyWidget,
        title="Levy Flight",
        collapsible=False,
    ),
}


class ReactivePath(param.Parameterized):
    value = param.DataFrame()


state = ReactivePath()


@pn.depends(WalkSelectorWidget, watch=True)
def update_speed_enbled(trajectory_type: TrajectoryType):
    match trajectory_type:
        case "LF":
            SpeedWidget.disabled = True
        case _:
            SpeedWidget.disabled = False


@pn.depends(WalkSelectorWidget)
def display_trajectory_form(trajectory_type: TrajectoryType):
    return TrajectoryForm[trajectory_type]


@pn.depends(
    WalkSelectorWidget,
    StepsWidget,
    XPositionWidget,
    YPositionWidget,
    SpeedWidget,
    CauchyCoefficientWidget,
    AlphaLevyWidget,
)
def plot_trajectory(
    trajectory_type: TrajectoryType,
    steps: int = 0,
    x: int = 0,
    y: int = 0,
    speed: int = 1,
    cauchy: float = 0.7,
    alpha: float = 2,
):
    path = None
    match trajectory_type:
        case "BM":
            path = walk.brownian_motion_2d(steps, speed, (x, y))
        case "CRW":
            path = walk.correlated_random_walk_2d(steps, speed, (x, y), cauchy)
        case "LF":
            path = walk.levy_walk_2d(steps, (x, y), cauchy, alpha)

    fig = go.Figure()

    fig.add_trace(walk.to_scatter3d(path))

    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
    )

    state.value = path

    return pn.Column(
        pn.pane.Markdown(f"## Trajectory"),
        pn.pane.Plotly(fig),
    )


@pn.depends(state.param.value, MetricSelectorWidget)
def plot_metrics(path: pd.DataFrame, metric: MetricType):
    fig = go.Figure()

    match metric:
        case "Path Length":
            metric_values = walk.path_lengths(path)

            fig.add_trace(
                go.Scatter(
                    x=path.index,
                    y=np.cumsum(metric_values),
                    marker=dict(size=2),
                    line=dict(width=2),
                    mode="lines",
                )
            )

        case "Mean Square Displacements":
            metric_values = walk.mean_squared_displacement(path)
            fig.add_trace(
                go.Scatter(
                    x=path.index,
                    y=metric_values,
                    marker=dict(size=2),
                    line=dict(width=2),
                    mode="lines",
                )
            )

        case "Turning Angles":
            metric_values = walk.turning_angles(path)

            fig.add_trace(
                go.Histogram(
                    x=metric_values,
                    nbinsx=100,
                    opacity=0.75,
                )
            )

    return pn.Column(
        pn.pane.Markdown(f"## Metrics"),
        MetricSelectorWidget,
        pn.pane.Plotly(fig),
    )


pn.template.MaterialTemplate(
    site="Random Walks",
    title="Trajectory and Metrics",
    sidebar=[pn.Column("Parameters", WalkSelectorWidget, display_trajectory_form)],
    main=[pn.Row(plot_trajectory, plot_metrics)],
).servable();